In [49]:
import synapseclient
from synapseclient import Project, File, Folder
from synapseclient import Schema, Column, Table, Row, RowSet, as_table_columns
import pandas as pd
import numpy as np

In [2]:
syn = synapseclient.Synapse()
syn.login(authToken=[redacted])

Welcome, Victor Baham!



#### Get individual and biospecimen metadata for the MSBB and ROSMAP cohorts from the NPS-AD study

In [46]:
nps_ad_indv = pd.read_csv(syn.get('syn55251012').path)
nps_ad_bios = pd.read_csv(syn.get('syn55251032').path)

nps_ad_gold = pd.merge(nps_ad_indv, nps_ad_bios, on="individualID")

ros_and_msbb_in_nps_indv = nps_ad_gold[(nps_ad_gold['cohort'] == 'MSBB') | (nps_ad_gold['cohort'] == 'ROSMAP')]
ros_and_msbb_in_nps_indv

,Component_x,individualID,cohort,species,sex,race,ethnicity,yearsEducation,apoeGenotype,diagnosis,...,nucleicAcidSource,cellType,samplingAge,samplingAgeUnits,sampleStatus,fastingState,visitNumber,assay,Id,entityId
0,IndividualHumanMetadataTemplate,AMPAD_MSSM_0000029668,MSBB,Human,male,Black or African American,Not Hispanic or Latino,NaN,33.0,NaN,...,single nucleus,NaN,NaN,NaN,frozen,NaN,NaN,snrnaSeq,337399ff-c8cd-4ee2-a87c-11488db02558,NaN
1,IndividualHumanMetadataTemplate,AMPAD_MSSM_00000M1061,MSBB,Human,male,Black or African American,Not Hispanic or Latino,NaN,33.0,NaN,...,single nucleus,NaN,NaN,NaN,frozen,NaN,NaN,snrnaSeq,bf1ad418-d4bc-492a-8426-b66c8fe7e9a2,NaN
2,IndividualHumanMetadataTemplate,AMPAD_MSSM_00000M1062,MSBB,Human,female,NaN,Hispanic or Latino,NaN,33.0,NaN,...,single nucleus,NaN,NaN,NaN,frozen,NaN,NaN,snrnaSeq,50757002-0330-441c-9e61-2b481494bb09,NaN
3,IndividualHumanMetadataTemplate,AMPAD_MSSM_0000010001,MSBB,Human,female,White,Not Hispanic or Latino,NaN,33.0,NaN,...,single nucleus,NaN,NaN,NaN,frozen,NaN,NaN,snrnaSeq,1593a784-36bf-449d-a476-2c6d29f11858,NaN
4,IndividualHumanMetadataTemplate,AMPAD_MSSM_0000010031,MSBB,Human,male,NaN,Hispanic or Latino,NaN,33.0,NaN,...,single nucleus,NaN,NaN,NaN,frozen,NaN,NaN,snrnaSeq,c2514bf6-2229-4a55-a3fe-65f7a673ddd1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489,IndividualHumanMetadataTemplate,R5508487,ROSMAP,Human,male,Black or African American,Not Hispanic or Latino,NaN,33.0,NaN,...,single nucleus,NaN,NaN,NaN,frozen,NaN,NaN,snrnaSeq,2b02ebb9-a0e9-4883-9e2e-7ba7350499f7,NaN
1490,IndividualHumanMetadataTemplate,R9380629,ROSMAP,Human,female,Black or African American,Not Hispanic or Latino,NaN,23.0,NaN,...,single nucleus,NaN,NaN,NaN,frozen,NaN,NaN,snrnaSeq,b9ad1423-925f-40b5-8429-943386b99ef2,NaN
1491,IndividualHumanMetadataTemplate,R5909439,ROSMAP,Human,female,Black or African American,Not Hispanic or Latino,NaN,44.0,NaN,...,single nucleus,NaN,NaN,NaN,frozen,NaN,NaN,snrnaSeq,10407afc-e3e9-4fea-8b37-044eab939981,NaN
1492,IndividualHumanMetadataTemplate,R8594936,ROSMAP,Human,female,Black or African American,Not Hispanic or Latino,NaN,33.0,NaN,...,single nucleus,NaN,NaN,NaN,frozen,NaN,NaN,snrnaSeq,a8ddcdd4-4be0-4200-bd3c-9a84228bfb6f,NaN


#### Get WGS assay metadata in MSBB and ROSMAP studies

In [23]:
msbb_wgs = pd.read_csv(syn.get('syn22360825').path)
rosmap_wgs = pd.read_csv(syn.get('syn21314542').path)

Downloaded syn22360825 to /home/jovyan/.synapseCache/526/75391526/MSBB_assay_wholeGenomeSeq_metadata.csv


Downloaded syn21314542 to /home/jovyan/.synapseCache/45/75338045/ROSMAP_assay_wholeGenomeSeq_metadata.csv


#### For MSBB samples, filter specimenID to last 5 characters

In [53]:
ros_and_msbb_in_nps_indv['specimenID'] = np.where(ros_and_msbb_in_nps_indv['cohort'] == 'MSBB', 
                                                  ros_and_msbb_in_nps_indv['specimenID'].str[-5:], ros_and_msbb_in_nps_indv['cohort'])

[WARNING] /tmp/ipykernel_1219739/378524507.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ros_and_msbb_in_nps_indv['specimenID'] = np.where(ros_and_msbb_in_nps_indv['cohort'] == 'MSBB',



In [54]:
msbb_hits = [str(x) for x in msbb_wgs['specimenID'].astype(str).tolist() if str(x) in ros_and_msbb_in_nps_indv['specimenID'].astype(str).tolist()]
rosmap_hits = [str(x) for x in rosmap_wgs['specimenID'].astype(str).tolist() if str(x) in ros_and_msbb_in_nps_indv['specimenID'].astype(str).tolist()]

In [55]:
rosmap_hits

[]

In [56]:
msbb_hits

['71719', '71734', '71789']